# 17장. 딥러닝을 이용한 자연어 처리

In [1]:
import numpy as np
import tensorflow as tf
from numpy import array
from tensorflow.keras.preprocessing.text import Tokenizer #단어빈도수
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding #Embedding임베딩

In [5]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence
text = '해보지 않으면 해낼 수 없다'

#해당 텍스트를 토큰화 합니다.
result = text_to_word_sequence(text)

print("\n원문:\n", text)
print("\n토큰화:\n", result)


원문:
 해보지 않으면 해낼 수 없다

토큰화:
 ['해보지', '않으면', '해낼', '수', '없다']


In [6]:
#전처리 하려는 세개의 문장 정하기
docs = ['먼저 텍스트의 각 단어를 나누어 토큰화 합니다.',
       '텍스트의 단어로 토큰화 해야 딥러닝에서 인식됩니다.',
       '토큰화 한 결과는 딥러닝에서 사용 할 수 있습니다.',
       ]

In [11]:
#토큰화 함수를 이용해 전처리 하는 과정
token = Tokenizer()
token.fit_on_texts(docs)
print(token.word_counts)

OrderedDict([('먼저', 1), ('텍스트의', 2), ('각', 1), ('단어를', 1), ('나누어', 1), ('토큰화', 3), ('합니다', 1), ('단어로', 1), ('해야', 1), ('딥러닝에서', 2), ('인식됩니다', 1), ('한', 1), ('결과는', 1), ('사용', 1), ('할', 1), ('수', 1), ('있습니다', 1)])


In [16]:
print("\n문장 카운트:\n", token.document_count) 
print("\n각 단어가 몇개의 문장에 포함되어 있는가:\n",token.word_docs)
print("\n각 단어에 매겨진 인덱스 값:\n",token.word_index)


문장 카운트:
 3

각 단어가 몇개의 문장에 포함되어 있는가:
 defaultdict(<class 'int'>, {'텍스트의': 2, '단어를': 1, '토큰화': 3, '합니다': 1, '나누어': 1, '먼저': 1, '각': 1, '인식됩니다': 1, '단어로': 1, '딥러닝에서': 2, '해야': 1, '할': 1, '사용': 1, '결과는': 1, '있습니다': 1, '한': 1, '수': 1})

각 단어에 매겨진 인덱스 값:
 {'토큰화': 1, '텍스트의': 2, '딥러닝에서': 3, '먼저': 4, '각': 5, '단어를': 6, '나누어': 7, '합니다': 8, '단어로': 9, '해야': 10, '인식됩니다': 11, '한': 12, '결과는': 13, '사용': 14, '할': 15, '수': 16, '있습니다': 17}


### 단어의 원 핫 인코딩
* 각단어를 0으로 바꾸어 주고 원하는 단어만 1로 바꾸어 주는 것
* 이를 수행하기 위해 먼저 단어 수만큼 0으로 채워진 백터 공간으로 바꿈

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer
text = '오랫동안 꿈꾸는 이는 그 꿈을 닮아간다'

token = Tokenizer()
token.fit_on_texts([text])
print(token.word_index)

{'오랫동안': 1, '꿈꾸는': 2, '이는': 3, '그': 4, '꿈을': 5, '닮아간다': 6}


In [21]:
x = token.texts_to_sequences([text])
print(x)

[[1, 2, 3, 4, 5, 6]]


In [23]:
from tensorflow.keras.utils import to_categorical

#인덱스 수에 하나를 추가해서 원 핫 인코딩 배열 만들기
word_size = len(token.word_index)+1
x = to_categorical(x, num_classes=word_size)
print(x)

[[[0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 1.]]]


### 단어 임베딩
* 원핫인코딩이 벡터의 길이가 너무 길어지니 공간적인 낭비가 크다. 그것을 해결하기 위한 방법이 단어 임베딩기법이다.<br>
* 유사도 계산

In [24]:
from tensorflow.keras.layers import Embedding

model = Sequential()
model.add(Embedding(16,4)) #입력될 총 단어수 16, 임베딩후 출력되는 벡터크기는 4
#(Embedding(16,4,input_length=2) 입력되는 단어수 16개중 2개씩 넣겠다

####  짧은 리뷰10개를 불러와 각각 긍정이면 1, 부정적이면 0이라는 클래스로 지정

In [26]:
# 텍스트 리뷰 자료를 지정합니다.
docs = ["너무 재밌네요","최고예요","참 잘 만든 영화예요","추천하고 싶은 영화입니다",
        "한번 더 보고싶네요","글쎄요","별로예요","생각보다 지루하네요","연기가 어색해요","재미없어요"]

# 긍정 리뷰는 1, 부정 리뷰는 0으로 클래스를 지정
classes = array([1,1,1,1,1,0,0,0,0,0])

In [27]:
#토큰화
token = Tokenizer()
token.fit_on_texts(docs)
print(token.word_index)

{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로예요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}


In [34]:
#인덱스로 새로운 배열 생성
x = token.texts_to_sequences(docs)
print("\n리뷰 텍스트, 토큰화 결과:\n", x)


리뷰 텍스트, 토큰화 결과:
 [[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13], [14], [15], [16, 17], [18, 19], [20]]


In [30]:
#서로 다른 길이의 데이터를 4로 맞추기(패딩과정:길이를 똑같이 맞춤)
padded_x = pad_sequences(x,4)
print(padded_x)

[[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [ 0 11 12 13]
 [ 0  0  0 14]
 [ 0  0  0 15]
 [ 0  0 16 17]
 [ 0  0 18 19]
 [ 0  0  0 20]]


In [42]:
#전체 단어의 맨 앞에 0이 먼저 나와야 하므로 총 단어 수에 1을 더함(인덱스있으니)
word_size = len(token.word_index)+1
print(word_size) #아까 단어가 20개였는데 21개 됨.

21


In [43]:
#4개의 단어가 들어가게 설정하면 임베딩 과정은 한줄로 표현
Embedding(word_size, 8, input_length=4)
#8은 임의로 정함. word_size만큼 입력값을 이용8개의 임베딩 결과를 만듦
#4개의 길이로 맞춰 주었으므로 4개의 단어가 들어가게 설정하면 

In [40]:
#단어 임베딩을 포함하여 딥러닝 모델을 만들고 결과를 출력
model = Sequential()
model.add(Embedding(word_size, 8, input_length=4))
model.add(Flatten()) #1차원으로
model.add(Dense(1, activation='sigmoid')) #출력값 1개 (긍정인지 부정인지이니)
model.compile(optimizer='adam', loss='binary_crossentropy', #0아니면1이니
             metrics=['accuracy'])
model.fit(padded_x, classes, epochs=20)
print("\n Accuracy: %.4f" % (model.evaluate(padded_x, classes)[1]))

Epoch 1/20
1/1 [==============================] - 0s 246ms/step - loss: 0.6950 - accuracy: 0.5000
Epoch 2/20
1/1 [==============================] - 0s 2ms/step - loss: 0.6929 - accuracy: 0.5000
Epoch 3/20
1/1 [==============================] - 0s 3ms/step - loss: 0.6907 - accuracy: 0.5000
Epoch 4/20
1/1 [==============================] - 0s 9ms/step - loss: 0.6886 - accuracy: 0.5000
Epoch 5/20
1/1 [==============================] - 0s 8ms/step - loss: 0.6865 - accuracy: 0.6000
Epoch 6/20
1/1 [==============================] - 0s 6ms/step - loss: 0.6844 - accuracy: 0.6000
Epoch 7/20
1/1 [==============================] - 0s 8ms/step - loss: 0.6823 - accuracy: 0.8000
Epoch 8/20
1/1 [==============================] - 0s 3ms/step - loss: 0.6801 - accuracy: 0.9000
Epoch 9/20
1/1 [==============================] - 0s 3ms/step - loss: 0.6780 - accuracy: 0.9000
Epoch 10/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6759 - accuracy: 0.9000
Epoch 11/20
1/1 [====================